# IRC0218 (z = 1.62) Cluster & Field D4000 Comparison Study 
## D4000 vs. Sersic Index, Effective Radius, Mass, Stellar Surface Mass Density 
# by Jenn Cooper 
# Last modified 5 July 2018

In [70]:
run /Users/jennifercooper/bootstrap_err.py

In [71]:
fieldn(x,y,yer,lcb1,lcb2,lcb3,ucb1,ucb2,ucb3)

NameError: name 'x' is not defined

In [19]:
ircn(x,y,yer,lcb1,lcb2,lcb3,ucb1,ucb2,ucb3)

NameError: name 'x' is not defined

In [20]:
fieldr(x,y,yer,lcb1,lcb2,lcb3,ucb1,ucb2,ucb3)

NameError: name 'x' is not defined

In [21]:
ircr(x,y,yer,lcb1,lcb2,lcb3,ucb1,ucb2,ucb3)

NameError: name 'x' is not defined

In [22]:
ircssmd(x,y,yer,lcb1,lcb2,lcb3,ucb1,ucb2,ucb3)

NameError: name 'x' is not defined

In [24]:
ircssmd_all(x,y,yer,lcb1,lcb2,lcb3,ucb1,ucb2,ucb3,xcb1)

NameError: name 'x' is not defined

In [29]:
test(x)

NameError: name 'x' is not defined

In [73]:
run ~/Documents/Gal_Ev/D4000_size/test.py

In [74]:
happyBirthdayEmily()

Happy Birthday to you!
Happy Birthday to you!
Happy Birthday, dear Emily.
Happy Birthday to you!


In [75]:
fieldn(x,y,yer,lcb1,lcb2,lcb3,ucb1,ucb2,ucb3)

NameError: name 'x' is not defined

In [76]:
import pyfits 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import astropy 
from astropy import cosmology 
import math as mt
from scipy.integrate import quad
from scipy.stats import chi2_contingency
from pylab import *
from scipy.optimize import curve_fit
import scipy as sp
import scipy.special
import scipy.stats as stats
import seaborn as sns 
import pandas as pd
from astropy.modeling.models import Sersic1D
from numpy import * 
import bces.bces
import nmmn.stats

#https://github.com/rsnemmen/BCES/blob/master/stats%20howto%20bces.ipynb

#field data (_f)
f = open('/Users/jennifercooper/Documents/Gal_Ev/D4000_size/field_size_n.txt', 'r')
lines = f.readlines()[1:]
f.close()

#create arrays 
f_re_er = [] 
f_re    = []
f_n     = []
f_n_er  = []
f_z     = []
f_dk    = []
f_lmass = []
f_flag  = []
f_ans_re_1 =[]
f_dk_p  = []
f_dk_n = []
f_re_kpca = []
f_ssmd = []

#pull array column 
for line in lines: 
    a = line.split()
    f_z.append(float(a[16]))
    f_re.append(float(a[6]))
    f_re_er.append(float(a[7]))
    f_n.append(float(a[8]))
    f_n_er.append(float(a[9]))
    f_dk.append(float(a[32]))
    f_lmass.append(float(a[21]))
    f_flag.append(float(a[36]))
    f_dk_p.append(float(a[33]))
    f_dk_n.append(float(a[34]))
    f_re_kpca.append(float(a[37]))
    f_ssmd.append(float(a[38]))

#assign arrays 
f_re    = np.array(f_re)
f_re_er = np.array(f_re_er)
f_n     = np.array(f_n)
f_n_er  = np.array(f_n_er)
f_z     = np.array(f_z)
f_dk    = np.array(f_dk)
f_lmass = np.array(f_lmass)
f_flag  = np.array(f_flag) #greater than 0.9 (1) is SF, less than is Q
f_dk_p  = np.array(f_dk_p)
f_dk_n  = np.array(f_dk_n)
f_re_kpca = np.array(f_re_kpca)
f_dk_err = (f_dk_p + f_dk_n)/2. #average of errors 
f_ssmd  = np.array(f_ssmd)


f_re_kpc = f_re*8.615 #arc to kpc
f_b = 1.9992*f_n - 0.3271 #value of b



In [81]:
def fieldn(x,y,yer,lcb1,lcb2,lcb3,ucb1,ucb2,ucb3):
    x  = f_n[np.where(f_flag>0.9)]
    y = f_dk[np.where(f_flag>0.9)]
    yer = f_dk_err[np.where(f_flag>0.9)]
    sort = np.argsort(x)
    x = x[sort]
    x = np.array(x)
    y = y[sort] 
    y = np.array(y)
    yer = yer[sort]
    yer = np.array(yer)
    xer=zeros(len(x))
    cov=zeros(len(x))   # no correlation between error measurements
    ax=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
    ax = subplot(2,2,1)
    ax = xlim(0.4,4)
    ax = ylim(-1,5)
    i = 0
    nboot=10000   # number of bootstrapping trials
    def func(x): return x[1]*x[0]+x[2]
    a,b,aerr,berr,covab=bces.bces.bcesp(x,xer,y,yer,cov,nboot)
    ybces=a[3]*x+b[3]  # the integer corresponds to the desired BCES method for plotting (3-ort, 0-y|x, 1-x|y, *don't use bissector*)
    # array with best-fit parameters
    fitm=np.array([ a[i],b[i] ])
    # covariance matrix of parameter uncertainties
    covm=np.array([ (aerr[i]**2,covab[i]), (covab[i],berr[i]**2) ])
    # Gets lower and upper bounds on the confidence band 
    lcb1,ucb1,x=nmmn.stats.confband(x, y, a[i], b[i], 0.68, x)
    lcb2,ucb2,x2=nmmn.stats.confband(x, y, a[i], b[i], 0.95, x)
    lcb3,ucb3,x3=nmmn.stats.confband(x, y, a[i], b[i], 0.997, x)
    errorbar(x,y,yerr=yer,fmt='o')
    ax = plot(x,ybces,'-k')
    ax = fill_between(x, lcb1, ucb1, alpha=0.6, facecolor='purple')
    ax = fill_between(x, lcb2, ucb2, alpha=0.3, facecolor='blue')
    ax = fill_between(x, lcb3, ucb3, alpha=0.4, facecolor='grey')
    #ax = bar(x,y,yerr=[f_dk_p[np.where(f_flag>0.9)], f_dk_n[np.where(f_flag>0.9)]], facecolor='none')
    ax = legend(loc='best')
    ax = xlabel('Field Sersic Index SF')
    ax = ylabel('$D4000$')
    
    ax = subplot(2,2,2)
    ax = xlim(0.4,5)
    ax = ylim(-1,5)
    x, y = pd.Series(f_n[np.where(f_flag<0.9)], name="Field Sersic Index Q"), pd.Series(f_dk[np.where(f_flag<0.9)], name="D4000")
    yer = pd.Series(f_dk_err[np.where(f_flag<0.9)])
    sort = np.argsort(x)
    x = x[sort]
    x = np.array(x)
    y = y[sort] 
    y = np.array(y)
    yer = yer[sort]
    yer = np.array(yer)
    xer=zeros(len(x))
    cov=zeros(len(x))   # no correlation between error measurements
    i = 0
    nboot=10000   # number of bootstrapping trials
    def func(x): return x[1]*x[0]+x[2]
    a,b,aerr,berr,covab=bces.bces.bcesp(x,xer,y,yer,cov,nboot)
    ybces=a[3]*x+b[3]  # the integer corresponds to the desired BCES method for plotting (3-ort, 0-y|x, 1-x|y, *don't use bissector*)
    # array with best-fit parameters
    fitm=np.array([ a[i],b[i] ])
    # covariance matrix of parameter uncertainties
    covm=np.array([ (aerr[i]**2,covab[i]), (covab[i],berr[i]**2) ])
    # Gets lower and upper bounds on the confidence band 
    lcb1,ucb1,x=nmmn.stats.confband(x, y, a[i], b[i], 0.68, x)
    lcb2,ucb2,x2=nmmn.stats.confband(x, y, a[i], b[i], 0.95, x)
    lcb3,ucb3,x3=nmmn.stats.confband(x, y, a[i], b[i], 0.997, x)
    errorbar(x,y,yerr=yer,fmt='o')
    ax = plot(x,ybces,'-k')
    ax = fill_between(x, lcb1, ucb1, alpha=0.6, facecolor='purple')
    ax = fill_between(x, lcb2, ucb2, alpha=0.3, facecolor='blue')
    ax = fill_between(x, lcb3, ucb3, alpha=0.4, facecolor='grey')
    ax = xlabel('Field Sersic Index Q')
    purple_patch = mpatches.Patch(color='purple', label='68%')
    blue_patch = mpatches.Patch(color='blue', label='95%', alpha = 0.3)
    grey_patch = mpatches.Patch(color='grey', label='99.7%', alpha=0.4)
    plt.legend(handles=[purple_patch,blue_patch,grey_patch])
    
    
    
    ax = subplot(2,2,3)
    ax = xlim(0.4,4)
    ax = ylim(-1,5)
    x, y = pd.Series(f_n[np.where(np.logical_and(f_flag>0.9,f_lmass>10.20))], name="Field Sersic Index SF Mass Complete"), pd.Series(f_dk[np.where(np.logical_and(f_flag>0.9,f_lmass>10.20))], name="D4000")
    yer = pd.Series(f_dk_err[np.where(np.logical_and(f_flag>0.9,f_lmass>10.20))])
    sort = np.argsort(x)
    x = x[sort]
    x = np.array(x)
    y = y[sort] 
    y = np.array(y)
    yer = yer[sort]
    yer = np.array(yer)
    xer=zeros(len(x))
    cov=zeros(len(x))   # no correlation between error measurements
    i = 0
    nboot=10000   # number of bootstrapping trials
    def func(x): return x[1]*x[0]+x[2]
    a,b,aerr,berr,covab=bces.bces.bcesp(x,xer,y,yer,cov,nboot)
    ybces=a[3]*x+b[3]  # the integer corresponds to the desired BCES method for plotting (3-ort, 0-y|x, 1-x|y, *don't use bissector*)
    # array with best-fit parameters
    fitm=np.array([ a[i],b[i] ])
    # covariance matrix of parameter uncertainties
    covm=np.array([ (aerr[i]**2,covab[i]), (covab[i],berr[i]**2) ])
    # Gets lower and upper bounds on the confidence band 
    lcb1,ucb1,x=nmmn.stats.confband(x, y, a[i], b[i], 0.68, x)
    lcb2,ucb2,x2=nmmn.stats.confband(x, y, a[i], b[i], 0.95, x)
    lcb3,ucb3,x3=nmmn.stats.confband(x, y, a[i], b[i], 0.997, x)
    errorbar(x,y,yerr=yer,fmt='o')
    ax = plot(x,ybces,'-k')
    ax = fill_between(x, lcb1, ucb1, alpha=0.6, facecolor='purple')
    ax = fill_between(x, lcb2, ucb2, alpha=0.3, facecolor='blue')
    ax = fill_between(x, lcb3, ucb3, alpha=0.4, facecolor='grey')
    ax = xlabel('Field Sersic Index SF Mass Complete')
    ax = ylabel('D4000')
    ax = plt.show()

In [82]:
fieldn(x,y,yer,lcb1,lcb2,lcb3,ucb1,ucb2,ucb3)

NameError: name 'x' is not defined